In [1]:
import pickle
import os
import sys
from datasets import load_dataset, Dataset
from ast import literal_eval
import pandas as pd
import faiss
import numpy as np
from typing import List, Optional, Dict, Tuple, Union
from tqdm.auto import tqdm
from FlagEmbedding import BGEM3FlagModel
from collections import defaultdict
import glob

In [2]:
class HybridRetriever:
    """
    BGE-M3 기반 하이브리드 리트리버
    - Dense: FAISS 인덱스
    - Sparse: BM25 pickle
    - Fusion: RRF 또는 Weighted
    """
    
    def __init__(
        self,
        model_name: str = 'BAAI/bge-m3',
        use_fp16: bool = True,
        data_path: str = '../Jang',
        fusion_method: str = 'rrf',
        rrf_k: int = 60,
        weights: Optional[Dict[str, float]] = None
    ):
        """
        Args:
            model_name: BGE-M3 모델 이름
            use_fp16: FP16 사용 여부
            data_path: 데이터 파일들이 있는 경로
            fusion_method: 'rrf' 또는 'weighted'
            rrf_k: RRF 상수 (보통 60)
            weights: weighted fusion 시 가중치 {'dense': 0.5, 'sparse': 0.5}
        """
        print("Initializing BGE-M3 Model...")
        self.model = BGEM3FlagModel(model_name, use_fp16=use_fp16)
        self.data_path = data_path
        self.fusion_method = fusion_method
        self.rrf_k = rrf_k
        
        if weights is None:
            self.weights = {'dense': 0.5, 'sparse': 0.5}
        else:
            self.weights = weights
        
        # 데이터 저장소
        self.dense_index: Optional[faiss.Index] = None
        self.sparse_vecs: Optional[List[Dict]] = None
        self.meta_df: Optional[pd.DataFrame] = None
        
        # 파일 로드
        self._load_files()
    
    def _load_files(self):
        """필요한 파일들 로드"""
        # 1. Dense Index 로드
        index_path = os.path.join(self.data_path, 'wikipedia_bge_m3.index')
        if os.path.exists(index_path):
            print(f"Loading Dense Index: {index_path}")
            self.dense_index = faiss.read_index(index_path)
            print(f"  → Loaded {self.dense_index.ntotal} vectors")
        else:
            raise FileNotFoundError(f"Dense index not found: {index_path}")
        
        # 2. Metadata 로드
        meta_path = os.path.join(self.data_path, 'wikipedia_chunks_meta.parquet')
        if os.path.exists(meta_path):
            print(f"Loading Metadata: {meta_path}")
            self.meta_df = pd.read_parquet(meta_path)
            print(f"  → Loaded {len(self.meta_df)} documents")
        else:
            raise FileNotFoundError(f"Metadata not found: {meta_path}")
        
        # 3. Sparse Vectors 로드
        sparse_parts_dir = os.path.join(self.data_path, 'wikipedia_sparse_parts')
        file_pattern = os.path.join(sparse_parts_dir, "wiki_sparse_part_*.pkl")
        part_files = sorted(
            glob.glob(file_pattern),
            key=lambda x: int(x.split('_')[-1].split('.')[0])
        )
        
        if part_files:
            print(f"Loading Sparse Vectors (Parts): {sparse_parts_dir}")
            print(f"  → 발견된 파일: {len(part_files)}개")
            
            self.sparse_vecs = []
            for file_path in part_files:
                with open(file_path, 'rb') as f:
                    data = pickle.load(f)
                    self.sparse_vecs.extend(data)
            
            print(f"  → Loaded {len(self.sparse_vecs)} sparse vectors")
        
        # 검증
        if len(self.meta_df) != len(self.sparse_vecs):
            print(f"[WARNING] Metadata({len(self.meta_df)}) and Sparse({len(self.sparse_vecs)}) count mismatch!")
        if self.dense_index.ntotal != len(self.meta_df):
            print(f"[WARNING] Dense Index({self.dense_index.ntotal}) and Metadata({len(self.meta_df)}) count mismatch!")
    
    def _compute_sparse_score(self, query_weights: Dict, doc_weights: Dict) -> float:
        """Sparse 점수 계산"""
        score = 0.0
        for token, q_weight in query_weights.items():
            if token in doc_weights:
                score += q_weight * doc_weights[token]
        return score
    
    # ========================================
    # 외부 사용 가능한 Dense 검색
    # ========================================
    def retrieve_dense(
        self, 
        query: str, 
        top_k: int = 5,
        only_scores: bool = False
    ) -> Union[pd.DataFrame, List[Tuple[int, float]]]:
        """
        Dense 검색만 수행 (외부 호출 가능)
        
        Args:
            query: 검색 쿼리
            top_k: 반환할 문서 수
            return_scores: True면 DataFrame 반환, False면 (idx, score) 리스트 반환
        
        Returns:
            return_scores=True: 검색 결과 DataFrame
            return_scores=False: [(idx, score), ...] 리스트
        """
        # 쿼리 임베딩
        q_vec = self.model.encode(
            [query],
            batch_size=1,
            max_length=8192,
            return_dense=True,
            return_sparse=False,
            return_colbert_vecs=False
        )['dense_vecs']
        
        # 정규화 및 검색
        q_vec = q_vec.astype('float32')
        faiss.normalize_L2(q_vec)
        scores, indices = self.dense_index.search(q_vec, top_k)
        
        # (index, score) 튜플 리스트 생성
        results = []
        for idx, score in zip(indices[0], scores[0]):
            if idx >= 0:  # 유효한 인덱스만
                results.append((int(idx), float(score)))
        
        # 스코어만 반환
        if only_scores:
            return results
        
        # DataFrame으로 변환하여 반환
        df_results = []
        for rank, (idx, score) in enumerate(results, start=1):
            if idx < len(self.meta_df):
                doc = self.meta_df.iloc[idx]
                df_results.append({
                    'rank': rank,
                    'score': score,
                    'doc_id': doc.get('doc_id', idx),
                    'title': doc.get('title', ''),
                    'text': doc.get('text', '')
                })
        
        return pd.DataFrame(df_results)
    
    # ========================================
    # 외부 사용 가능한 Sparse 검색
    # ========================================
    def retrieve_sparse(
        self, 
        query: str, 
        top_k: int = 5,
        only_scores: bool = False
    ) -> Union[pd.DataFrame, List[Tuple[int, float]]]:
        """
        Sparse 검색만 수행 (외부 호출 가능)
        
        Args:
            query: 검색 쿼리
            top_k: 반환할 문서 수
            return_scores: True면 DataFrame 반환, False면 (idx, score) 리스트 반환
        
        Returns:
            return_scores=True: 검색 결과 DataFrame
            return_scores=False: [(idx, score), ...] 리스트
        """
        # 쿼리 임베딩
        query_output = self.model.encode(
            [query],
            return_dense=False,
            return_sparse=True,
            return_colbert_vecs=False
        )
        query_sparse = query_output['lexical_weights'][0]
        
        # 전체 문서에 대해 점수 계산
        all_scores = [
            self._compute_sparse_score(query_sparse, doc_vec)
            for doc_vec in self.sparse_vecs
        ]
        
        # 상위 k개 추출
        top_indices = np.argsort(all_scores)[::-1][:top_k]
        
        results = []
        for idx in top_indices:
            results.append((int(idx), float(all_scores[idx])))
        
        # 스코어만 반환
        if only_scores:
            return results
        
        # DataFrame으로 변환하여 반환
        df_results = []
        for rank, (idx, score) in enumerate(results, start=1):
            if idx < len(self.meta_df):
                doc = self.meta_df.iloc[idx]
                df_results.append({
                    'rank': rank,
                    'score': score,
                    'doc_id': doc.get('doc_id', idx),
                    'title': doc.get('title', ''),
                    'text': doc.get('text', '')
                })
        
        return pd.DataFrame(df_results)
    
    # ========================================
    # 내부 헬퍼 함수들 (기존 호환성 유지)
    # ========================================
    def _retrieve_dense(self, query: str, k: int) -> List[Tuple[int, float]]:
        """내부용 Dense 검색 (return_scores=False와 동일)"""
        return self.retrieve_dense(query, top_k=k, return_scores=False)
    
    def _retrieve_sparse(self, query: str, k: int) -> List[Tuple[int, float]]:
        """내부용 Sparse 검색 (return_scores=False와 동일)"""
        return self.retrieve_sparse(query, top_k=k, return_scores=False)
    
    def _rrf_fusion(
        self,
        dense_results: List[Tuple[int, float]],
        sparse_results: List[Tuple[int, float]]
    ) -> List[Tuple[int, float]]:
        """RRF(Reciprocal Rank Fusion) 적용"""
        rrf_scores: Dict[int, float] = defaultdict(float)
        
        # Dense 순위 반영
        for rank, (idx, _) in enumerate(dense_results, start=1):
            rrf_scores[idx] += 1.0 / (self.rrf_k + rank)
        
        # Sparse 순위 반영
        for rank, (idx, _) in enumerate(sparse_results, start=1):
            rrf_scores[idx] += 1.0 / (self.rrf_k + rank)
        
        # 점수 기준 정렬
        sorted_items = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
        return [(idx, score) for idx, score in sorted_items]
    
    def _weighted_fusion(
        self,
        dense_results: List[Tuple[int, float]],
        sparse_results: List[Tuple[int, float]]
    ) -> List[Tuple[int, float]]:
        """가중치 기반 점수 결합"""
        # 점수 정규화
        def normalize(results):
            if not results:
                return {}
            scores = [s for _, s in results]
            min_s, max_s = min(scores), max(scores)
            if max_s - min_s == 0:
                return {idx: 1.0 for idx, _ in results}
            return {idx: (s - min_s) / (max_s - min_s) for idx, s in results}
        
        dense_norm = normalize(dense_results)
        sparse_norm = normalize(sparse_results)
        
        # 가중치 적용
        final_scores: Dict[int, float] = defaultdict(float)
        for idx, score in dense_norm.items():
            final_scores[idx] += self.weights['dense'] * score
        for idx, score in sparse_norm.items():
            final_scores[idx] += self.weights['sparse'] * score
        
        # 정렬
        sorted_items = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
        return [(idx, score) for idx, score in sorted_items]
    
    # ========================================
    # 하이브리드 검색 (기존)
    # ========================================
    def retrieve_hybrid(
        self,
        query: str,
        top_k: int = 5,
        dense_k: int = 100,
        sparse_k: int = 100
    ) -> pd.DataFrame:
        """
        하이브리드 검색 (Dense + Sparse + Fusion)
        
        Args:
            query: 검색 쿼리
            top_k: 최종 반환 문서 수
            dense_k: Dense에서 가져올 후보 수
            sparse_k: Sparse에서 가져올 후보 수
        
        Returns:
            검색 결과 DataFrame
        """
        # Dense & Sparse 검색
        dense_results = self._retrieve_dense(query, dense_k)
        sparse_results = self._retrieve_sparse(query, sparse_k)
        
        # Fusion
        if self.fusion_method == 'rrf':
            fused_results = self._rrf_fusion(dense_results, sparse_results)
        elif self.fusion_method == 'weighted':
            fused_results = self._weighted_fusion(dense_results, sparse_results)
        else:
            raise ValueError(f"Unknown fusion method: {self.fusion_method}")
        
        # Top-K 추출 및 메타데이터 결합
        results = []
        for rank, (idx, score) in enumerate(fused_results[:top_k], start=1):
            if idx < len(self.meta_df):
                doc = self.meta_df.iloc[idx]
                results.append({
                    'rank': rank,
                    'score': score,
                    'doc_id': doc.get('doc_id', idx),
                    'title': doc.get('title', ''),
                    'text': doc.get('text', '')
                })
        
        return pd.DataFrame(results)
    
    # 기존 retrieve_single을 retrieve_hybrid로 변경 (호환성)
    def retrieve_single(self, *args, **kwargs) -> pd.DataFrame:
        """retrieve_hybrid의 별칭 (하위 호환성)"""
        return self.retrieve_hybrid(*args, **kwargs)
    
    def retrieve_batch(
        self,
        question_df: pd.DataFrame,
        paragraph_col: str = 'paragraph',
        question_col: str = 'question',
        n: Optional[int] = None,
        top_k: int = 5,
        dense_k: int = 100,
        sparse_k: int = 100,
        seed: int = 42,
        mode: str = 'hybrid'  # ← 추가: 'hybrid', 'dense', 'sparse'
    ) -> pd.DataFrame:
        """
        DataFrame의 모든 질문에 대해 검색 수행
        
        Args:
            question_df: 질문 DataFrame
            paragraph_col: paragraph 컬럼명
            question_col: question 컬럼명
            n: 샘플링 개수 (None이면 전체)
            top_k: 각 질문당 반환할 문서 수
            dense_k: Dense 후보 수 (hybrid 모드에만 사용)
            sparse_k: Sparse 후보 수 (hybrid 모드에만 사용)
            seed: 랜덤 시드
            mode: 'hybrid', 'dense', 'sparse' 중 선택
        
        Returns:
            확장된 DataFrame (각 질문당 top_k개 행 생성)
        """
        # 샘플링
        if n is None:
            samples = question_df.copy()
        else:
            real_n = min(n, len(question_df))
            samples = question_df.sample(n=real_n, random_state=seed).copy()
        
        print(f"\n{len(samples)}개의 질문에 대해 Top-{top_k} {mode.upper()} 검색을 수행합니다.")
        print(f"예상 결과 행: {len(samples) * top_k}개\n")
        
        merged_results = []
        
        for idx, row in tqdm(samples.iterrows(), total=len(samples), desc="Searching"):
            # 쿼리 생성
            query = f"{row[paragraph_col]} \n\n {row[question_col]}"
            
            # 모드에 따라 검색
            if mode == 'dense':
                results = self.retrieve_dense(query, top_k=top_k, only_scores=True)
            elif mode == 'sparse':
                results = self.retrieve_sparse(query, top_k=top_k, only_scores=True)
            elif mode == 'hybrid':
                # Dense & Sparse 검색
                dense_results = self._retrieve_dense(query, dense_k)
                sparse_results = self._retrieve_sparse(query, sparse_k)
                
                # Fusion
                if self.fusion_method == 'rrf':
                    fused_results = self._rrf_fusion(dense_results, sparse_results)
                elif self.fusion_method == 'weighted':
                    fused_results = self._weighted_fusion(dense_results, sparse_results)
                else:
                    raise ValueError(f"Unknown fusion method: {self.fusion_method}")
                
                results = fused_results[:top_k]
            else:
                raise ValueError(f"Unknown mode: {mode}. Use 'hybrid', 'dense', or 'sparse'")
            
            # Top-K 결과 처리
            for rank, (doc_idx, score) in enumerate(results, start=1):
                if doc_idx >= len(self.meta_df):
                    continue
                
                # 원본 데이터 복사
                combined_row = row.to_dict()
                
                # 검색 결과 추가
                doc = self.meta_df.iloc[doc_idx]
                combined_row['ctx_rank'] = rank
                combined_row['ctx_score'] = score
                combined_row['ctx_id'] = doc.get('doc_id', doc_idx)
                combined_row['ctx_title'] = doc.get('title', '')
                combined_row['ctx_text'] = doc.get('text', '')
                
                merged_results.append(combined_row)
        
        final_df = pd.DataFrame(merged_results)
        print(f"\n완료! 총 {len(final_df)}개의 행이 생성되었습니다.")
        
        return final_df

In [3]:
ROOT_DIR = '/data/ephemeral/pro-nlp-generationfornlp-nlp-13'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
dataset = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))

In [4]:
# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
df = pd.DataFrame(records)

In [5]:
df["choices_len"] = df["choices"].apply(len)
df4 = df[df['choices_len'] == 4]

In [6]:
hybrid_retriever = HybridRetriever('BAAI/bge-m3')

Initializing BGE-M3 Model...


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading Dense Index: ../Jang/wikipedia_bge_m3.index
  → Loaded 1165197 vectors
Loading Metadata: ../Jang/wikipedia_chunks_meta.parquet
  → Loaded 1165197 documents
Loading Sparse Vectors (Parts): ../Jang/wikipedia_sparse_parts
  → 발견된 파일: 232개
  → Loaded 1165197 sparse vectors


In [7]:
df_dense = hybrid_retriever.retrieve_batch(df4, n=10, mode='dense')


10개의 질문에 대해 Top-5 DENSE 검색을 수행합니다.
예상 결과 행: 50개



Searching:   0%|          | 0/10 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 393.54it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 58.86it/s]



완료! 총 50개의 행이 생성되었습니다.


In [32]:
df_dense.head(50)

,id,paragraph,question,choices,answer,question_plus,choices_len,ctx_rank,ctx_score,ctx_id,ctx_title,ctx_text
0,generation-for-nlp-1261,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",다음 중 위 노래에 영감을 준 사건은?,"[아우스부르크의 평화, 스페인 왕위 계승 전쟁, 낭트칙령, 30년 전쟁]",4,None,4,1,0.571697,75492,프리드리히 5세 폰 데어 팔츠,보헤미아 국왕시절의 베드르지흐(프리드리히 5세) '''프리드리히 5세'''(Frie...
1,generation-for-nlp-1261,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",다음 중 위 노래에 영감을 준 사건은?,"[아우스부르크의 평화, 스페인 왕위 계승 전쟁, 낭트칙령, 30년 전쟁]",4,None,4,2,0.569315,26124,보리스 고두노프 (오페라),"위에 있는 지도를 페오도르에게 펼쳐 보이며 ""이 넓은 러시아의 영토가 마침내 너의 ..."
2,generation-for-nlp-1261,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",다음 중 위 노래에 영감을 준 사건은?,"[아우스부르크의 평화, 스페인 왕위 계승 전쟁, 낭트칙령, 30년 전쟁]",4,None,4,3,0.558124,91712,페르디난트 1세 (오스트리아),상적인 섭정정치를 이루었고 이는 오스트리아 제국을 혼란에 빠트려 1846년부터 18...
3,generation-for-nlp-1261,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",다음 중 위 노래에 영감을 준 사건은?,"[아우스부르크의 평화, 스페인 왕위 계승 전쟁, 낭트칙령, 30년 전쟁]",4,None,4,4,0.554392,33280,30년 전쟁,쪽|200px|백산 전투의 재현 보헤미아 분쟁은 여전히 지역적인 분란으로 남아있었다...
4,generation-for-nlp-1261,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",다음 중 위 노래에 영감을 준 사건은?,"[아우스부르크의 평화, 스페인 왕위 계승 전쟁, 낭트칙령, 30년 전쟁]",4,None,4,5,0.551977,10287,황제 찬가,"Franz, den Kaiser, :Unsern guten Kaiser Franz!..."
5,generation-for-nlp-1063,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,"[제1차 세계대전, 뉴딜, 냉전, 대공황]",1,None,4,1,0.660926,16294,표현의 자유,파일:자유권.png|섬네일|4가지 자유사상과 양심의 자유표현과 언론의 자유집회와 결...
6,generation-for-nlp-1063,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,"[제1차 세계대전, 뉴딜, 냉전, 대공황]",1,None,4,2,0.636950,478196,빌라왈 부토 자르다리,. === 표현의 자유 === 민주주의의 독실한 옹호자인 빌라왈은 검열을 반복적으로...
7,generation-for-nlp-1063,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,"[제1차 세계대전, 뉴딜, 냉전, 대공황]",1,None,4,3,0.627735,163547,네 가지 자유,'''네 가지 자유'''(Four Freedoms)는 프랭클린 D. 루스벨트가 미국...
8,generation-for-nlp-1063,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,"[제1차 세계대전, 뉴딜, 냉전, 대공황]",1,None,4,4,0.625068,310580,영구 평화론,1차 세계 대전 초기에 웰스는 프로이센 군국주의와 독재가 대중 정부로 대체되면 유럽...
9,generation-for-nlp-1063,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,"[제1차 세계대전, 뉴딜, 냉전, 대공황]",1,None,4,5,0.617348,74299,민주화,". 그 결과 사회는 집단적 위험을 인식하면 독재정권, 독재정권의 방향으로 발전할 것..."


In [31]:
df_sparse = hybrid_retriever.retrieve_batch(df4, n=10, mode='sparse')


10개의 질문에 대해 Top-5 SPARSE 검색을 수행합니다.
예상 결과 행: 50개



Searching:   0%|          | 0/10 [00:00<?, ?it/s]


완료! 총 50개의 행이 생성되었습니다.


In [33]:
df_sparse.head(50)

,id,paragraph,question,choices,answer,question_plus,choices_len,ctx_rank,ctx_score,ctx_id,ctx_title,ctx_text
0,generation-for-nlp-1261,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",다음 중 위 노래에 영감을 준 사건은?,"[아우스부르크의 평화, 스페인 왕위 계승 전쟁, 낭트칙령, 30년 전쟁]",4,None,4,1,0.295703,776,5월 30일,'''5월 30일'''은 그레고리력으로 150번째(윤년일 경우 151번째) 날에 해...
1,generation-for-nlp-1261,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",다음 중 위 노래에 영감을 준 사건은?,"[아우스부르크의 평화, 스페인 왕위 계승 전쟁, 낭트칙령, 30년 전쟁]",4,None,4,2,0.287463,75492,프리드리히 5세 폰 데어 팔츠,보헤미아 국왕시절의 베드르지흐(프리드리히 5세) '''프리드리히 5세'''(Frie...
2,generation-for-nlp-1261,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",다음 중 위 노래에 영감을 준 사건은?,"[아우스부르크의 평화, 스페인 왕위 계승 전쟁, 낭트칙령, 30년 전쟁]",4,None,4,3,0.277344,465805,이라크의 역사,이라크는 티그리스강과 유프라테스강을 따라 위치한 비옥한 초승달 지대의 가장 동쪽에 ...
3,generation-for-nlp-1261,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",다음 중 위 노래에 영감을 준 사건은?,"[아우스부르크의 평화, 스페인 왕위 계승 전쟁, 낭트칙령, 30년 전쟁]",4,None,4,4,0.265826,5392,독일의 국가,년까지 공식행사에서 자주 사용되었다. 이 노래의 선율은 하이든이 1796~1797년...
4,generation-for-nlp-1261,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",다음 중 위 노래에 영감을 준 사건은?,"[아우스부르크의 평화, 스페인 왕위 계승 전쟁, 낭트칙령, 30년 전쟁]",4,None,4,5,0.256428,282327,옛 나라 목록,"'''옛 나라 목록'''에서는 인류 역사상 과거에 존재했던 국가, 국제 기구 또는 ..."
5,generation-for-nlp-1063,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,"[제1차 세계대전, 뉴딜, 냉전, 대공황]",1,None,4,1,0.332873,319845,4명의 경찰관,"파일:Cairo conference.jpg|섬네일|중국 총통 장제스, 미국 대통령 ..."
6,generation-for-nlp-1063,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,"[제1차 세계대전, 뉴딜, 냉전, 대공황]",1,None,4,2,0.332186,163547,네 가지 자유,'''네 가지 자유'''(Four Freedoms)는 프랭클린 D. 루스벨트가 미국...
7,generation-for-nlp-1063,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,"[제1차 세계대전, 뉴딜, 냉전, 대공황]",1,None,4,3,0.330513,69171,세계 평화,파일:Peace symbol (fixed width).svg|섬네일|평화 기호로도 ...
8,generation-for-nlp-1063,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,"[제1차 세계대전, 뉴딜, 냉전, 대공황]",1,None,4,4,0.324940,16294,표현의 자유,파일:자유권.png|섬네일|4가지 자유사상과 양심의 자유표현과 언론의 자유집회와 결...
9,generation-for-nlp-1063,하지만 전 우리 역사의 이 특별한 순간 시민의 자유가 중요함을 누구보다 더 잘 알고...,엘레노어 루즈벨트는 연설에서 시민의 자유에 먼저 위협이 된 것이 무엇이라고 하였는가?,"[제1차 세계대전, 뉴딜, 냉전, 대공황]",1,None,4,5,0.323332,453,1월 6일,'''1월 6일'''은 그레고리력으로 6번째 날에 해당한다. * 1540년 - 영국...


In [34]:
def analyze_retrieval_consistency(df_dense, df_sparse, top_k=5, n_samples=10):
    """
    top_k: 한 샘플당 리트리브된 문서 개수
    n_samples: 전체 샘플(질문) 개수
    """
    total_overlap = 0
    results = []

    print(f"--- 샘플별 리트리버 일치도 분석 (Sample: {n_samples}개, Top-k: {top_k}) ---")

    for i in range(n_samples):
        # 각 샘플의 구간 계산
        start_idx = i * top_k
        end_idx = start_idx + top_k
        
        # 각 샘플에 해당하는 문서 셋 추출
        dense_set = set(df_dense['ctx_text'].iloc[start_idx:end_idx])
        sparse_set = set(df_sparse['ctx_text'].iloc[start_idx:end_idx])
        
        # 교집합 계산
        intersection = dense_set.intersection(sparse_set)
        overlap_count = len(intersection)
        total_overlap += overlap_count
        
        results.append(overlap_count)
        
        # 개별 샘플 결과 출력 (선택 사항)
        
        print(f"샘플 {i+1:2d}: {overlap_count}/{top_k} 개 일치")

    # 종합 통계
    avg_overlap = total_overlap / n_samples
    print("-" * 40)
    print(f"전체 평균 일치 개수: {avg_overlap:.2f} / {top_k}")
    print(f"전체 일치율(Average Overlap Rate): {(avg_overlap/top_k)*100:.1f}%")

# 사용 예시
analyze_retrieval_consistency(df_dense, df_sparse, top_k=5, n_samples=10)

--- 샘플별 리트리버 일치도 분석 (Sample: 10개, Top-k: 5) ---
샘플  1: 1/5 개 일치
샘플  2: 2/5 개 일치
샘플  3: 1/5 개 일치
샘플  4: 1/5 개 일치
샘플  5: 0/5 개 일치
샘플  6: 2/5 개 일치
샘플  7: 0/5 개 일치
샘플  8: 3/5 개 일치
샘플  9: 2/5 개 일치
샘플 10: 0/5 개 일치
----------------------------------------
전체 평균 일치 개수: 1.20 / 5
전체 일치율(Average Overlap Rate): 24.0%
